In [1]:
import fiiscraper as fscp
from fiiscraper.models import FII
import pandas as pd
import yfinance as yf
from datetime import date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import requests

scraper = fscp.Scraper()

# Adiciona o sufixo .SA para ativos brasileiros
lista_fiis = scraper.listar_todos_fiis()
lista_reduzida = lista_fiis[:10]

#ticker = ["BCFF11", "YYYY11", "VISC11"]
df = scraper.buscar_precos_em_lote([fii.ticker for fii in lista_reduzida])

# Busca os dados de indicadores do dia
indicadores_fiis = []
for fii in lista_reduzida:

    # Busca indicadores do FII
    indicadores_fii = scraper.buscar_indicadores_dia(fii.ticker)
    
    # Adiciona FII à lista
    indicadores_fiis.append(indicadores_fii)

In [2]:
indicadores_fiis[0].div_yield

0.0661

In [2]:
import boto3
import pandas as pd
import io  # Necessário para o buffer em memória
from datetime import date, timedelta

df = pd.DataFrame({'A':[1, 2, 3],
                   'B': [4, 5, 6]})

today = date.today()
yesterday = today - timedelta(days=1)

nome_bucket = 'fii-data-bucket'
nome_arquivo_s3_indicators = f'raw/pdaily_prices/price_date={yesterday.isoformat()}/data_parquet'
nome_arquivo_s3_prices = f'raw/price_history_snapshots/ingest_date={today.isoformat()}/data_parquet'

s3_client = boto3.client('s3')

# 1. Cria um buffer de bytes em memória (uma "caixa de despacho" virtual)
buffer_parquet = io.BytesIO()

# 2. Escreve o DataFrame no buffer em formato Parquet
df.to_parquet(buffer_parquet, index=False)

# 3. "Rebobina" o buffer para o início antes de o ler para o upload
buffer_parquet.seek(0)

# 4. Usa put_object para enviar o conteúdo do buffer (os bytes)
s3_client.put_object(
    Bucket=nome_bucket,
    Key=nome_arquivo_s3_indicators,
    Body=buffer_parquet
)

s3_client.put_object(
    Bucket=nome_bucket,
    Key=nome_arquivo_s3_indicators,
    Body=buffer_parquet
)

NoCredentialsError: Unable to locate credentials

,A,B
0,1,4
1,2,5
2,3,6
